In [1]:
import numpy as np
import torch

In [2]:
[np.clip(np.random.uniform(-1.0, 1.0), -1.0, 1.0)] 

[0.35877600322341796]

In [3]:
from drlagent_off_policy_agent import OUNoise

noise = OUNoise(action_space=2, max_sigma=0.1, min_sigma=0.1, decay_period=8000000)

In [4]:
n = noise.get_noise()
n

array([ 0.03000605, -0.03365393])

In [5]:
action = [np.random.uniform(-1.0, 1.0) for _ in range(2)]
action

[-0.08145742739137596, -0.3252408284638033]

In [6]:
torch.add(torch.tensor(action), torch.tensor(n))

tensor([-0.0515, -0.3589], dtype=torch.float64)

In [7]:
from collections import deque

d = deque(maxlen=5)

for i in range(10):
    d.append(i)
    print(d)

deque([0], maxlen=5)
deque([0, 1], maxlen=5)
deque([0, 1, 2], maxlen=5)
deque([0, 1, 2, 3], maxlen=5)
deque([0, 1, 2, 3, 4], maxlen=5)
deque([1, 2, 3, 4, 5], maxlen=5)
deque([2, 3, 4, 5, 6], maxlen=5)
deque([3, 4, 5, 6, 7], maxlen=5)
deque([4, 5, 6, 7, 8], maxlen=5)
deque([5, 6, 7, 8, 9], maxlen=5)
